In [3]:
%run ../../common_utils.py

In [4]:
from catboost import CatBoostRegressor
import optuna
from optuna.samplers import TPESampler
from IPython.utils import io
import unidecode
from catboost import Pool

In [5]:
random_state = 1
number_of_splits = 5

In [6]:
def load_data(val_data=False,path=None,cat_unknown=False):
    train, test, metadata = load_all_data(path=path)
    nonCategorical, categorical = get_cat_and_non_cat_data(metadata)
    all_features = list(train.columns)
    all_features.remove('price')
    numerical_features = ['area_total','area_kitchen','area_living','floor','rooms','ceiling',
        'bathrooms_shared','bathrooms_private','balconies','loggias','phones','building_id','constructed','stories']
    categorical_to_numerical(train, ['street','address'])
    categorical_to_numerical(test, ['street','address'])
    if not val_data:
        X_train, y_train, test_labels = pre_process_numerical(features = all_features, numerical_features = numerical_features, train = train, test = test,
                            outliers_value=7, val_data=val_data, val_split=0.2, random_state=42, scaler="std",
                            add_R="True", add_rel_height="True", droptable=[],
                            one_hot_encode=False, cat_features=categorical, drop_old=True,categorical_unknown=cat_unknown)
        y_train_log = np.log(y_train)
        return X_train, y_train, y_train_log, test_labels
    else:
        X_train, y_train, X_test, y_test, test_labels = pre_process_numerical(features = all_features, numerical_features = numerical_features, train = train, test = test,
                            outliers_value=7, val_data=val_data, val_split=0.2, random_state=42, scaler="std",
                            add_R="True", add_rel_height="True", droptable=[],
                            one_hot_encode=False, cat_features=categorical, drop_old=True,categorical_unkown=cat_unknown)
        y_train_log = np.log(y_train)
        return X_train, y_train, y_train_log, X_test, y_test, test_labels

In [11]:
%%capture --no-display
X_train, y_train, y_train_log, test_labels = load_data(path='../',cat_unknown=True)
selected_features_catboost=['building_id','area_kitchen', 'area_living',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'phones',
       'new', 'street', 'address', 'seller', 'layout', 'condition', 'district',
       'constructed', 'stories', 'elevator_without', 'elevator_passenger',
       'material', 'parking', 'heating', 'r', 'rel_height']

X_train = X_train[selected_features_catboost]
_, _, metadata = load_all_data(path='../')
nonCategorical, categorical = get_cat_and_non_cat_data(metadata)
text_features=['street','address']
categorical.append('street')
categorical.append('address')
X_train = X_train.astype({'street':'int','address':'int','seller':'int','layout':'int','condition':'int','district':'int','material':'int','parking':'int','heating':'int'})
# X_train.drop(['area_kitchen', 'area_living', 'rooms', 'ceiling',
#        'bathrooms_shared', 'bathrooms_private', 'windows_court', 
#        'windows_street', 'balconies', 'loggias', 'phones', 'new', 'constructed',
#        'stories', 'elevator_without', 'elevator_passenger',
#        'r', 'rel_height'],axis=1,inplace=True)

In [12]:
def objective(trial,X_train,y_train):
    
    param = {
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-1),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-2, 1e0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 10),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 20),
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 20),  
    }
    # Conditional Hyper-Parameters
    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    scores = []
        
    cv = GroupKFold(n_splits=number_of_splits)
    groups = X_train["building_id"]
    i = 1
    for train_index, test_index in cv.split(X_train, y_train, groups):
        print("starting on split ",i)
        i+=1
        X_train2, X_test = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train2, y_test = y_train.iloc[train_index], y_train.iloc[test_index]
        X_train2.drop(['building_id'], axis=1,inplace=True)
        X_test.drop(['building_id'], axis=1,inplace=True)

        model = CatBoostRegressor(**param, random_state=random_state,loss_function='RMSE', cat_features=categorical)
        
        model.fit(
            X_train2,
            y_train2,
            eval_set=[(X_test, y_test)],
            verbose=False,
            early_stopping_rounds=100,
        )
        
        prediction = np.exp(model.predict(X_test))
        score = root_mean_squared_log_error(prediction, np.exp(y_test))
        scores.append(score)
    return np.average(scores)

In [13]:
study = optuna.create_study(sampler=TPESampler(), direction="minimize")
func = lambda trial: objective(trial, X_train, y_train_log)
study.optimize(func, n_trials=10, timeout=600) # Run for 10 minutes
print("Number of completed trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("\tBest Score: {}".format(trial.value))
print("\tBest Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-11-11 20:05:12,647] A new study created in memory with name: no-name-0dec1979-f8df-443a-b53e-7f708e68656a
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
Custom logger is already specified. Specify more than one logger at same time is not thread safe.

starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-11 20:07:40,749] Trial 0 finished with value: 0.29659492153632294 and parameters: {'learning_rate': 0.011178116746795775, 'l2_leaf_reg': 0.019924670924834312, 'colsample_bylevel': 0.08691455386202702, 'depth': 9, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 4, 'one_hot_max_size': 9, 'subsample': 0.4112845599243803}. Best is trial 0 with value: 0.29659492153632294.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-11 20:12:30,090] Trial 1 finished with value: 0.8104071649816392 and parameters: {'learning_rate': 8.317233032433974e-05, 'l2_leaf_reg': 0.08153465717513544, 'colsample_bylevel': 0.0704133559521057, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 3, 'one_hot_max_size': 3}. Best is trial 0 with value: 0.29659492153632294.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


...

starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-11 20:14:25,749] Trial 6 finished with value: 0.8315974243004197 and parameters: {'learning_rate': 0.00010881074058495042, 'l2_leaf_reg': 0.01681702944440864, 'colsample_bylevel': 0.06136170238984698, 'depth': 1, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 4, 'one_hot_max_size': 3, 'subsample': 0.8455339112036674}. Best is trial 0 with value: 0.29659492153632294.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-11 20:15:47,761] Trial 7 finished with value: 0.738145874291905 and parameters: {'learning_rate': 0.00029134776365406643, 'l2_leaf_reg': 0.01862158993648414, 'colsample_bylevel': 0.09894013854180753, 'depth': 6, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'min_data_in_leaf': 8, 'one_hot_max_size': 4, 'bagging_temperature': 6.400790173459352}. Best is trial 0 with value: 0.29659492153632294.


Number of completed trials: 8
Best trial:
	Best Score: 0.29659492153632294
	Best Params: 
    learning_rate: 0.011178116746795775
    l2_leaf_reg: 0.019924670924834312
    colsample_bylevel: 0.08691455386202702
    depth: 9
    boosting_type: Ordered
    bootstrap_type: Bernoulli
    min_data_in_leaf: 4
    one_hot_max_size: 9
    subsample: 0.4112845599243803


In [14]:
study.optimize(func, n_trials=20)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-11 20:16:06,206] Trial 8 finished with value: 0.8289608922325501 and parameters: {'learning_rate': 0.00010953439623100823, 'l2_leaf_reg': 0.16285307499985685, 'colsample_bylevel': 0.042148496228111995, 'depth': 2, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 4, 'one_hot_max_size': 13}. Best is trial 0 with value: 0.29659492153632294.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-11 20:17:31,125] Trial 9 finished with value: 0.35339202466352615 and parameters: {'learning_rate': 0.007094017586029255, 'l2_leaf_reg': 0.04756366013319338, 'colsample_bylevel': 0.041342933962715996, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 19, 'one_hot_max_size': 2}. Best is trial 0 with value: 0.29659492153632294.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


...

[I 2021-11-11 20:35:54,796] Trial 25 finished with value: 0.28312913442745763 and parameters: {'learning_rate': 0.0154995116732571, 'l2_leaf_reg': 0.022500324281349942, 'colsample_bylevel': 0.09996254245773571, 'depth': 9, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 11, 'one_hot_max_size': 11, 'subsample': 0.5026629502438935}. Best is trial 16 with value: 0.27076959290908187.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-11 20:37:25,829] Trial 26 finished with value: 0.2700280258709413 and parameters: {'learning_rate': 0.04493315459969882, 'l2_leaf_reg': 0.1016956128187997, 'colsample_bylevel': 0.09068864619114643, 'depth': 8, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 16, 'one_hot_max_size': 5, 'subsample': 0.708857568242697}. Best is trial 26 with value: 0.2700280258709413.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-11 20:37:47,839] Trial 27 finished with value: 0.3403218489333818 and parameters: {'learning_rate': 0.04205281677290802, 'l2_leaf_reg': 0.11660462201681147, 'colsample_bylevel': 0.024943387477795335, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 19, 'one_hot_max_size': 5, 'subsample': 0.7627109103996765}. Best is trial 26 with value: 0.2700280258709413.


In [15]:
print("Number of completed trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("\tBest Score: {}".format(trial.value))
print("\tBest Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of completed trials: 28
Best trial:
	Best Score: 0.2700280258709413
	Best Params: 
    learning_rate: 0.04493315459969882
    l2_leaf_reg: 0.1016956128187997
    colsample_bylevel: 0.09068864619114643
    depth: 8
    boosting_type: Ordered
    bootstrap_type: Bernoulli
    min_data_in_leaf: 16
    one_hot_max_size: 5
    subsample: 0.708857568242697


In [16]:
study.optimize(func, n_trials=100)

starting on split  1


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-11 20:43:28,410] Trial 28 finished with value: 0.4098248694534686 and parameters: {'learning_rate': 0.0024085396369145293, 'l2_leaf_reg': 0.15092143087290114, 'colsample_bylevel': 0.09014458586036572, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 15, 'one_hot_max_size': 5, 'subsample': 0.8496164750232961}. Best is trial 26 with value: 0.2700280258709413.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


...

[I 2021-11-12 00:37:27,353] Trial 125 finished with value: 0.27153202261161713 and parameters: {'learning_rate': 0.034198845785439295, 'l2_leaf_reg': 0.014209752870383893, 'colsample_bylevel': 0.0948452361163803, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 20, 'one_hot_max_size': 8}. Best is trial 113 with value: 0.26552493946336475.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-12 00:41:05,234] Trial 126 finished with value: 0.2712201922701414 and parameters: {'learning_rate': 0.04667166325768196, 'l2_leaf_reg': 0.017755175020918137, 'colsample_bylevel': 0.08873085214142296, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 18, 'one_hot_max_size': 4}. Best is trial 113 with value: 0.26552493946336475.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-12 00:43:22,552] Trial 127 finished with value: 0.28487658807838667 and parameters: {'learning_rate': 0.0809812600411995, 'l2_leaf_reg': 0.015569108227175834, 'colsample_bylevel': 0.09641583276792941, 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'min_data_in_leaf': 19, 'one_hot_max_size': 2, 'bagging_temperature': 7.6255673159290005}. Best is trial 113 with value: 0.26552493946336475.


In [17]:
print("Number of completed trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("\tBest Score: {}".format(trial.value))
print("\tBest Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of completed trials: 128
Best trial:
	Best Score: 0.26552493946336475
	Best Params: 
    learning_rate: 0.0595682504013377
    l2_leaf_reg: 0.01619654653510247
    colsample_bylevel: 0.08580608350061232
    depth: 10
    boosting_type: Ordered
    bootstrap_type: MVS
    min_data_in_leaf: 18
    one_hot_max_size: 7


In [18]:
study.optimize(func, n_trials=100)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-12 00:45:38,856] Trial 128 finished with value: 0.2780080146335905 and parameters: {'learning_rate': 0.027597537998304892, 'l2_leaf_reg': 0.023083064242951118, 'colsample_bylevel': 0.08428497075377861, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 15, 'one_hot_max_size': 9}. Best is trial 113 with value: 0.26552493946336475.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-12 00:48:57,129] Trial 129 finished with value: 0.7432157445342015 and parameters: {'learning_rate': 0.00024858764987969236, 'l2_leaf_reg': 0.010987588744587804, 'colsample_bylevel': 0.08659441937380498, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 17, 'one_hot_max_size': 6}. Best is trial 113 with value: 0.26552493946336475.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


...

[I 2021-11-12 03:08:42,626] Trial 225 finished with value: 0.2645287060295985 and parameters: {'learning_rate': 0.051859941577656804, 'l2_leaf_reg': 0.034649950459367, 'colsample_bylevel': 0.09831783649004837, 'depth': 10, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 20, 'one_hot_max_size': 16}. Best is trial 139 with value: 0.26295902459609144.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-12 03:10:00,516] Trial 226 finished with value: 0.2648538079444914 and parameters: {'learning_rate': 0.05321213039425935, 'l2_leaf_reg': 0.0345304167790445, 'colsample_bylevel': 0.0969405244069298, 'depth': 10, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 20, 'one_hot_max_size': 16}. Best is trial 139 with value: 0.26295902459609144.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-12 03:11:16,174] Trial 227 finished with value: 0.26650210267606517 and parameters: {'learning_rate': 0.07289044225803289, 'l2_leaf_reg': 0.03683343598265032, 'colsample_bylevel': 0.09700824608701578, 'depth': 10, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 20, 'one_hot_max_size': 15}. Best is trial 139 with value: 0.26295902459609144.


In [19]:
print("Number of completed trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("\tBest Score: {}".format(trial.value))
print("\tBest Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of completed trials: 228
Best trial:
	Best Score: 0.26295902459609144
	Best Params: 
    learning_rate: 0.09983491174888588
    l2_leaf_reg: 0.013644264455059599
    colsample_bylevel: 0.09788222400849068
    depth: 8
    boosting_type: Ordered
    bootstrap_type: MVS
    min_data_in_leaf: 19
    one_hot_max_size: 7


In [20]:
study.optimize(func, n_trials=1000)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-12 03:12:18,536] Trial 228 finished with value: 0.2698131856124367 and parameters: {'learning_rate': 0.0849424895578963, 'l2_leaf_reg': 0.03924550780360866, 'colsample_bylevel': 0.09627335194249229, 'depth': 10, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'min_data_in_leaf': 20, 'one_hot_max_size': 16, 'bagging_temperature': 0.29204539723933554}. Best is trial 139 with value: 0.26295902459609144.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-12 03:13:33,739] Trial 229 finished with value: 0.26548849283234743 and parameters: {'learning_rate': 0.05587716868936805, 'l2_leaf_reg': 0.034625786802720056, 'colsample_bylevel': 0.09805290394391818, 'depth': 10, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 19, 'one_hot_max_size': 14}. Best is trial 139 with value: 0.26295902459609144.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


...

[I 2021-11-12 07:39:02,170] Trial 462 finished with value: 0.26292092232942565 and parameters: {'learning_rate': 0.09992491472163195, 'l2_leaf_reg': 0.42030533031687556, 'colsample_bylevel': 0.09171389443304014, 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 15, 'one_hot_max_size': 11}. Best is trial 437 with value: 0.2596557027450385.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


[I 2021-11-12 07:40:18,039] Trial 463 finished with value: 0.2639677027686576 and parameters: {'learning_rate': 0.08476497269495922, 'l2_leaf_reg': 0.35212478166337896, 'colsample_bylevel': 0.09136535195877407, 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 15, 'one_hot_max_size': 11}. Best is trial 437 with value: 0.2596557027450385.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


starting on split  1
starting on split  2
starting on split  3
starting on split  4
starting on split  5


KeyboardInterrupt: 